# Package and Data Import

In [1]:
import sys 
sys.version

'3.8.6 (v3.8.6:db455296be, Sep 23 2020, 13:31:39) \n[Clang 6.0 (clang-600.0.57)]'

In [5]:
import numpy as np
from src import DataImport
import matplotlib.pyplot as plt
from src import Models
import shap
import pandas as pd 
import xgboost
from pathlib import Path
from sklearn.model_selection import StratifiedKFold, cross_val_score
from src import Evaluate

In [6]:
! pip list

Package                  Version
------------------------ --------
anyio                    3.6.2
appnope                  0.1.3
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
arrow                    1.2.3
astor                    0.8.1
asttokens                2.2.1
attrs                    22.2.0
autograd                 1.5
autograd-gamma           0.5.0
backcall                 0.2.0
beautifulsoup4           4.11.2
bleach                   6.0.0
cffi                     1.15.1
cloudpickle              2.2.1
comm                     0.1.2
contourpy                1.0.7
cycler                   0.11.0
debugpy                  1.6.6
decorator                5.1.1
defusedxml               0.7.1
executing                1.2.0
fastjsonschema           2.16.2
fonttools                4.38.0
formulaic                0.5.2
fqdn                     1.5.1
future                   0.18.3
graphlib-backport        1.0.3
idna                     3.4
importlib-metadata       6.0.

# dfMort imputed on 22th Feb

## Imputation

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

df=pd.read_csv(Path.cwd().parent/'Data/HRS/data_preprocess/Data/merge_data_step_3.csv')
df['death'] = [0 if np.isnan(x) else 1 for x in df['deathYear']]
df=df.dropna(thresh=50)

df_to_missing_forest=df.drop(columns=['death','deathYear','deathMonth','deathReason','year_check','hhidpn','hhid','pn'])



In [ ]:
# missing forest
df_to_missing_forest=df.drop(columns=['death','deathYear','deathMonth','deathReason','year_check','hhidpn','hhid','pn'])
imputer = MissForest(n_estimators=500, max_iter=5)
dfLBImp = imputer.fit_transform(df_to_missing_forest)


In [ ]:
data_non_missing_version_2 = pd.DataFrame(columns=df_to_missing_forest.columns,data=dfLBImp)
data_non_missing_version_2.loc[:,['death','deathYear','deathMonth','deathReason','year_check','hhidpn','hhid','pn']]=df.loc[:,['death','deathYear','deathMonth','deathReason','year_check','hhidpn','hhid','pn']]

data_non_missing_version_2.to_csv(Path.cwd().parent/'Data/HRS/Bio_data/df_by_us_no_missing_based_on_merge_step_3',index=False)


## Performance check

In [ ]:
dfMort= pd.read_csv(Path.cwd()/'Data/HRS/Bio_data/df_by_us_no_missing_based_on_merge_step_3.csv')

dfMort['eversmokeYN'] = dfMort['eversmokeYN']*-1
binary_columns = [x for x in dfMort.columns if 'YN' in x]
for column in binary_columns:
    dfMort[column] = [1 if x >=0 else -1 for x in dfMort[column]]

dfMort.rename(columns={'deathYear': 'death_year', 'deathMonth': 'death_month'}, inplace=True)
dfMort['death'].replace({np.nan: 0}, inplace=True)
dfMort['deathYR'] = dfMort['death_year'] + dfMort['death_month'] / 12
dfMort = dfMort.loc[dfMort['age'] >= 52,]
dfMort.shape
for model_selection in ['xgb','lgb']:
    print(f'\n{model_selection}')
    model = Models.Model_fixed_test_size(data=dfMort, test_size=0.3,domain_list=domains['all'], model=model_selection,train_subset_size=1, order=0, y_colname='death')
    evas = Evaluate.metric(model)
    print(f'imv={evas.imv},\nroc-auc={evas.auc_score},\npr-auc={evas.pr_auc},\nf1={evas.pr_f1},\nefron_r2={evas.efron_rsquare},\nffc_r2={evas.ffc_r2},\nIP={evas.pr_no_skill}')
    

In [ ]:
# check value_counts difference 

#column = 'Zanxiety'
column = 'rocc'
accumulated_counts = 0
temp=df[column].value_counts().sort_values(ascending=False)
print(f'in original dataset, value counts of var {column} is :\n{df_with_missing[column].value_counts()}')
print(f'in df, value counts of var {column} is :\n{df[column].value_counts()}')
cat_num=len(df_with_missing[column].value_counts())
df_temp=recode_categorical_vars(column,cat_num,df).copy()
print(df_temp[column].value_counts())

In [ ]:
# check NANs 

for column in df_to_missing_forest.columns:
    print(column,sum(df_to_missing_forest[column].isna()),df_to_missing_forest[column].dtype)

In [ ]:
filenames= 'merge_data.csv,merge_data_non_standardise.csv,merge_data_non_standardise_before.csv,merge_data_selected_author_rows_no_missing.csv,merge_data_selected_author_rows_no_missing_versioin_1.csv,merge_data_selected_author_rows_no_missing_versioin_2.csv,merge_data_selected_author_rows_no_missing_versioin_3.csv,merge_data_selected_int_year.csv,merge_data_step_3.csv'.split(',')

for filename in filenames:
    print(filename)
    df_with_missing=pd.read_csv(Path.cwd().parent/f'Data/HRS/data_preprocess/Data/{filename}')
    print(df_with_missing.rocc.value_counts())

# Functions 

In [ ]:
def recode_categorical_vars(column,cat_num,df):
    
    # pair of values and their number of observations 
    cat_val_count_lst=[]
    count=0
    for index,value in df[column].value_counts().items():
        count +=1
        if count>=cat_num+1:
            break
        else:
            # print(index,value)
            cat_val_count_lst.append([index,value])
    # cut off calculation 
    cut_point={}
    for index in range(0,cat_num-1):
        cat_diff = cat_val_count_lst[index+1][0]-cat_val_count_lst[index][0]
        gravity = cat_val_count_lst[index][1]/(cat_val_count_lst[index][1]+cat_val_count_lst[index+1][1])
        cut_point[cat_val_count_lst[index][0]] = cat_val_count_lst[index][0]+gravity*cat_diff
    # cut bins 
    df[column]=pd.cut(df[column],bins=[cats[0]-100]+cats+[cats[len(cats)-1]+100],labels=[x[0] for x in cat_val_count_lst])
    return df
